In [1]:
import numpy as np
import cv2
import random

In [2]:
img = cv2.imread('scene6.jpg')
height = img.shape[0]
width = img.shape[1]

In [3]:
img_blob = cv2.dnn.blobFromImage(img, swapRB=True, crop=False)   # swapRB Because --> convert BGR to RGB

In [4]:
class_labels = ["person","bicycle","car","motorbike","aeroplane","bus","train","truck","boat","traffic light",
                "fire hydrant","street sign","stop sign","parking meter","bench","bird","cat","dog","horse","sheep",
                "cow","elephant","bear","zebra","giraffe","hat","backpack","umbrella","shoe","eye glasses","handbag",
                "tie","suitcase","frisbee","skis","snowboard","sports ball","kite","baseball bat","baseball glove",
                "skateboard","surfboard","tennis racket","bottle","plate","wine glass","cup","fork","knife","spoon",
                "bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut","cake",
                "chair","sofa","pottedplant","bed","mirror","diningtable","window","desk","toilet","door","tv",
                "laptop","mouse","remote","keyboard","cell phone","microwave","oven","toaster","sink","refrigerator",
                "blender","book","clock","vase","scissors","teddy bear","hair drier","toothbrush"]

In [5]:
# Red, Lime, Blue, Yellow, Cyan, Magenta, Silver, Gray, Maroon, Olive, Green, Purple, Teal, Navy
class_colors = ["255,0,0","0,255,0","0,0,255","255,255,0","0,255,255","255,0,255","192,192,192","128,128,128",
               "128,0,0","128,128,0","0,128,0","128,0,128","0,128,128","0,0,128"]

class_colors = [np.array(x.split(",")).astype("int") for x in class_colors]
class_colors = np.array(class_colors)

In [6]:
# Loading pretrained model from buffer model weights and buffer config files
maskrcnn = cv2.dnn.readNetFromTensorflow('maskrcnn_buffermodel.pb','maskrcnn_bufferconfig.txt')
maskrcnn.setInput(img_blob)

In [7]:
(boxes,masks)  = maskrcnn.forward(["detection_out_final","detection_masks"])
no_of_detections = boxes.shape[2]

In [8]:
for i in np.arange(0, no_of_detections):
    prediction_confidence = boxes[0, 0, i, 2]
    
    if prediction_confidence >= 0.30:
        
       
        predicted_class_index = int(boxes[0, 0, i, 1])
        predicted_class_label = class_labels[predicted_class_index]
        
        bounding_box = boxes[0, 0, i, 3:7] * np.array([width, height, width, height])
        (start_x, start_y, end_x, end_y) = bounding_box.astype("int")
        
        bounding_box_Width = end_x - start_x
        bounding_box_Height = end_y - start_y
        
        object_mask = masks[i, predicted_class_index]  # obtain the bounding mask co-ordinates
        object_mask = cv2.resize(object_mask, (bounding_box_Width, bounding_box_Height)) # resize mask to bounding_box_Width & Height
        
        # minimum threshold value to convert float based mask array to binary, if true respective values will be true.
        object_mask = (object_mask >= 0.3)

        roi = img[start_y:end_y, start_x:end_x]
        roi = roi[object_mask]  # slice the roi array based on the bounding mask
        
        mask_color = random.choice(class_colors) # get a random mask color from the numpy array of colors
        
        #add a transparent color cover to the region of interest
        roi_transparent_cover = ((0.3 * mask_color) + (0.5 * roi)).astype("uint8")
        img[start_y:end_y, start_x:end_x][object_mask] = roi_transparent_cover
        mask_color = [int(c) for c in mask_color] 
        
        
        predicted_class_label = "{}: {:.2f}%".format(class_labels[predicted_class_index], prediction_confidence * 100)
        
        cv2.rectangle(img, (start_x, start_y), (end_x, end_y), mask_color, 2)
        cv2.putText(img, predicted_class_label, (start_x, start_y -7), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (225,225,225), 1)


cv2.imshow("xyz", img)
cv2.waitKey(0)                 
cv2.destroyAllWindows()

---------------------------------------------